In [1]:
!pip install node2vec

  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
   ----------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import networkx as nx
import os
import xml.etree.ElementTree as ET
from node2vec import Node2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import nltk
import string
import spacy
from operator import itemgetter
nlp = spacy.load("en_core_web_sm")

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
directorio = "/content/gdrive/MyDrive/PLN Agosto - 2024/"

Mounted at /content/gdrive


In [7]:
def lee_xml_eat():
    lista_grafos = []
    G_freq=nx.Graph()
    G_asoc= nx.Graph()
    conjunto_nodos = set()
    tree = ET.parse(directorio+'eat-stimulus-response.xml')
    root = tree.getroot()
    todos = 8211
    i=0
    nodos={}
    total = 0
    j = 0
    for child in root:
        estimulo = child.get('word')
        total =child.get('all')
        estimulo = estimulo.lower()
        estimulo = estimulo.replace(" ","_")
        for response in child.iter('response'):
            frecuencia = response.get('n')
            asociacion = response.get('r')
            respuesta = response.get('word')
            respuesta = respuesta.lower()
            respuesta = respuesta.replace(" ","_")
            if respuesta not in conjunto_nodos:
                nodos[respuesta] = j
                G_freq.add_node(respuesta)
                G_asoc.add_node(respuesta)
                conjunto_nodos.add(respuesta)
                j+=1
            G_freq.add_edge(estimulo,respuesta,weight=float(1/int(frecuencia)))
            G_asoc.add_edge(estimulo,respuesta,weight=float(1-float(asociacion)))
    lista_grafos.append(G_freq)
    lista_grafos.append(G_asoc)
    print("Grafos creados")
    return lista_grafos

In [10]:
grafos = lee_xml_eat()
grafo_freq = grafos[0]
grafo_asoc = grafos[1]
print(grafo_freq.edges())
print("Total de nodos",grafo_freq.number_of_nodes())
print("Total de aristas",grafo_freq.number_of_edges())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
grafos = lee_xml_eat()
grafo_freq = grafos[0]
grafo_asoc = grafos[1]
node2vec = Node2Vec(grafo_freq,dimensions=300)
model = node2vec.fit(window=10,min_count=1,batch_words=4)
print("Guardando embeddings...")
model.wv.save_word2vec_format(directorio+"freq.emb")


Grafos creados


Computing transition probabilities:   0%|          | 0/23218 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
grafos = lee_xml_eat()
grafo_freq = grafos[0]
grafo_asoc = grafos[1]
node2vec = Node2Vec(grafo_asoc,dimensions=300)
model = node2vec.fit(window=10,min_count=1,batch_words=4)
print("Guardando embeddings...")
model.wv.save_word2vec_format(directorio+"asoc.emb")

Grafos creados


Computing transition probabilities:   0%|          | 0/23218 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [06:46<00:00, 40.64s/it]


KeyboardInterrupt: 

In [ ]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

wv = KeyedVectors.load_word2vec_format(directorio+"asoc.emb",binary=False)
total = wv["king"] - wv["man"] + wv["woman"]
print("Modelo cargado..")
print(wv.most_similar(positive=[total],topn=20))
total = wv["paris"] - wv["france"] + wv["spain"]
print("Modelo cargado..")
print(wv.most_similar(positive=[total],topn=20))

Modelo cargado..
[('king', 0.7085300087928772), ('woman', 0.5515092015266418), ('throne', 0.38645023107528687), ('majesty', 0.35767078399658203), ('andrews', 0.3566025197505951), ('princessling', 0.3476901054382324), ('abducted', 0.3460976779460907), ('the_queen', 0.34567052125930786), ('kingly', 0.34363603591918945), ('zonophone', 0.33773913979530334), ('pageantry', 0.3359178304672241), ('crown', 0.32742375135421753), ('liver_salt', 0.3254445195198059), ('monarch', 0.3246691823005676), ('liver_salts', 0.32320883870124817), ('queen', 0.3220468759536743), ('reigning', 0.31814107298851013), ('dethrone', 0.3175676763057709), ('robe', 0.3145774304866791), ('hong', 0.3141893148422241)]
Modelo cargado..
[('paris', 0.6295017600059509), ('spain', 0.6266786456108093), ('seine', 0.5125035047531128), ('portugal', 0.4084120988845825), ('flamenco', 0.39542192220687866), ('eiffel', 0.39391955733299255), ('bonn', 0.3825426697731018), ('sightseeing', 0.3759342133998871), ('nancy', 0.3752371668815613),

In [ ]:
wv = KeyedVectors.load_word2vec_format(directorio+"freq.emb",binary=False)
total = wv["king"] - wv["man"] + wv["woman"]
print("Modelo cargado..")
print(wv.most_similar(positive=[total],topn=20))
total = wv["paris"] - wv["france"] + wv["spain"]
print("Modelo cargado..")
print(wv.most_similar(positive=[total],topn=20))

Modelo cargado..
[('king', 0.7168864607810974), ('woman', 0.538760781288147), ('princess', 0.32140475511550903), ('bursar', 0.31035134196281433), ('st.', 0.30898547172546387), ('throne', 0.30777686834335327), ('robe', 0.30438467860221863), ('considerate', 0.30160751938819885), ('proclaim', 0.2894558310508728), ('hail', 0.28736114501953125), ('bong', 0.28625836968421936), ('hong', 0.28463783860206604), ('coat_of_arms', 0.2835005521774292), ('count', 0.28107282519340515), ('fourth', 0.2796558737754822), ('ladyship', 0.27715247869491577), ('majesty', 0.276675283908844), ('castle', 0.27507898211479187), ('vanquished', 0.2738201320171356), ('goldilocks', 0.27188840508461)]
Modelo cargado..
[('paris', 0.633418083190918), ('spain', 0.5816461443901062), ('seine', 0.5274801850318909), ('metro', 0.4552043676376343), ('bonn', 0.4483654499053955), ('eiffel', 0.3899242579936981), ('nancy', 0.37028324604034424), ('sara', 0.3398692309856415), ('bizarre', 0.3320724368095398), ('touring', 0.31560793519

In [ ]:
wv = KeyedVectors.load_word2vec_format(directorio+"asoc.emb",binary=False)
total = wv["insect"] + wv["honey"] + wv["stripes"] + wv["yellow"] + wv["black"]
total = total /5
print("Modelo cargado..")
print(wv.most_similar(total,topn=20))

Modelo cargado..
[('yellow', 0.6189721822738647), ('stripes', 0.5795342326164246), ('black', 0.5633469820022583), ('honey', 0.5447647571563721), ('insect', 0.5371432304382324), ('bumble-bee', 0.4837120771408081), ('tsetse', 0.4761532247066498), ('mosquitoes', 0.46379509568214417), ('insects', 0.46071210503578186), ('wasp', 0.4518756866455078), ('sting', 0.4446922540664673), ('stripe', 0.4429231286048889), ('caterpillar', 0.43826642632484436), ('mosquito', 0.4308270215988159), ('buzz', 0.4273469150066376), ('stung', 0.420186847448349), ('moth', 0.4174724519252777), ('colour', 0.4158550202846527), ('zebra', 0.4125511944293976), ('bugs', 0.41218772530555725)]


In [ ]:
wv.similarity("france","spain")

0.35162708